In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import KNNImputer
df=pd.read_csv("C:/Users/USER/Desktop/cars_data.csv")
df.shape[0]
df

We have 7777 rows of data that contain a lot of columns that are totally useless because all of its columns are NaN so we need to drop them

Drop All the rows that all of its data are NaN , After that we have reached to 6864 data rows instead of 7777 

In [ ]:
df=df.dropna(how='all')
df.shape[0]



In [ ]:
df.isna().any()
percentage_of_missing_data=df.isna().sum()/df.shape[0]
percentage_of_missing_data


Printing the Percentage of the missing data 


In [ ]:
percentage_of_missing_data=df.isna().sum()/df.shape[0]
percentage_of_missing_data.plot(kind='bar', color='skyblue', edgecolor='black')
plt.xlabel('Columns')
plt.ylabel('Percentage of Missing Data')
plt.title('Percentage of Missing Data in Each Feature')

plt.show()

In [ ]:
import re

def convert_to_numeric(data):
    numbers = re.findall(r'\d+', str(data))   
    return sum(map(int, numbers)) if numbers else np.nan

df['عدد الركاب'] = df['عدد الركاب'].apply(convert_to_numeric)
df

In [ ]:
df["عدد الركاب"].isna().sum()


In [ ]:
column_to_process = df[['عدد الركاب']]
model=KNNImputer(n_neighbors=1000)
new_data = model.fit_transform(column_to_process)
df['عدد الركاب'] = new_data
df['عدد الركاب'] = np.floor(df['عدد الركاب']).astype(int)

df


In [ ]:
car_size=df.groupby('عدد الركاب').size().reset_index(name="count")
car_size

In [ ]:
df["إضافات"]=df["إضافات"].fillna("")
df["فتحة سقف"]=df["إضافات"].apply(lambda x: 1 if "فتحة سقف" in x else 0)
df=df.drop(columns=["إضافات"])
df

Dropping the "معروضة" column

In [ ]:
df=df.drop(columns=["معروضة", "وسيلة الدفع"])
df